In [27]:
import re
import PyPDF2
import pandas as pd
import string
import re
import nltk

In [37]:
stopword = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()
ps = nltk.PorterStemmer()

In [38]:
f = open('101_german_conversation.txt',encoding='utf-8')
content = f.readlines()


In [45]:
text = []
for i in content:
    if i != '\n':
        text.append(i)

In [47]:
df = pd.DataFrame(text)
df.columns = ['Text']

In [53]:
df = df.iloc[1:1887]

In [56]:
def raw_text(m):
    if ': ' in m:
        p = m.split(': ')[0]
        m = m.split(': ')[1]
    if '\n' in m:
        m = m.split('\n')[0]
    return m

In [59]:
df['Raw_Text'] = df['Text'].apply(lambda x: raw_text(x.lower()))

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [ipykernel_launcher.py:1]


In [61]:
df.drop('Text',axis=1,inplace=True)

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [frame.py:4164]


In [62]:
df.head(10)

,Raw_Text
2,silke und julia machen ferien in berlin. sie h...
3,"guten morgen, silke! wie hast du geschlafen?"
4,sehr gut! und du?
5,auch sehr gut! was willst du heute unternehmen?
6,"hmmm\tich habe lust, auf einen markt zu gehen!"
7,super! ein wochenmarkt?
8,"nein, ich will auf einen antiquitätenmarkt gehen."
9,"klingt gut, kennst du einen?"
10,"nein, ich gucke mal im handy nach... hier ist ..."
11,„antikmarkt am ostbahnhof“


In [65]:
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

In [63]:
def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

In [67]:
df['tokenized'] = df['Raw_Text'].apply(lambda x: tokenize(remove_punct(x)))

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [ipykernel_launcher.py:1]


In [77]:
df['text'] = df['Raw_Text'].apply(lambda x: remove_punct(x))

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [ipykernel_launcher.py:1]


In [78]:
df.head(10)

,Raw_Text,tokenized,text
2,silke und julia machen ferien in berlin. sie h...,"[silke, und, julia, machen, ferien, in, berlin...",silke und julia machen ferien in berlin sie ha...
3,"guten morgen, silke! wie hast du geschlafen?","[guten, morgen, silke, wie, hast, du, geschlafen]",guten morgen silke wie hast du geschlafen
4,sehr gut! und du?,"[sehr, gut, und, du]",sehr gut und du
5,auch sehr gut! was willst du heute unternehmen?,"[auch, sehr, gut, was, willst, du, heute, unte...",auch sehr gut was willst du heute unternehmen
6,"hmmm\tich habe lust, auf einen markt zu gehen!","[hmmm, ich, habe, lust, auf, einen, markt, zu,...",hmmm\tich habe lust auf einen markt zu gehen
7,super! ein wochenmarkt?,"[super, ein, wochenmarkt]",super ein wochenmarkt
8,"nein, ich will auf einen antiquitätenmarkt gehen.","[nein, ich, will, auf, einen, antiquitätenmark...",nein ich will auf einen antiquitätenmarkt gehen
9,"klingt gut, kennst du einen?","[klingt, gut, kennst, du, einen]",klingt gut kennst du einen
10,"nein, ich gucke mal im handy nach... hier ist ...","[nein, ich, gucke, mal, im, handy, nach, hier,...",nein ich gucke mal im handy nach hier ist eine...
11,„antikmarkt am ostbahnhof“,"[, antikmarkt, am, ostbahnhof, ]",„antikmarkt am ostbahnhof“


In [71]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [81]:
from matplotlib import pyplot
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import numpy as np

In [101]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

def visualise_common_words_unigram(df,n=40):
    common_words = get_top_n_words(df['text'], n)
    df2 = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
    fig = px.bar(df2,x='ReviewText',y='count')
    fig.update_layout(
            height=800,
            title_text= 'Distribution of {} most used unigrams'.format(n),
            xaxis_title="Unigrams",
            yaxis_title="Frequency"
        )
    fig.show()
    return df2
    
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

def visualise_common_words_bigrams(df,n=40):
    common_words = get_top_n_bigram(df['text'], n)
    df4 = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
    
    fig = px.bar(df4,x='ReviewText',y='count')
    fig.update_layout(
            height=800,
            title_text= 'Distribution of {} most used bigrams'.format(n),
            xaxis_title="bigrams",
            yaxis_title="Frequency"
        )
    fig.show()
    return df4

    

In [102]:
unigrams = visualise_common_words_unigram(df,n=500)

In [103]:
bigrams = visualise_common_words_bigrams(df,500)

In [105]:
unigrams[unigrams['count'] >= 10]

,ReviewText,count
0,sie,432
1,die,416
2,ich,378
3,ist,309
4,der,270
...,...,...
232,frau,10
233,wollte,10
234,zuerst,10
235,drei,10


In [99]:
from HanTa import HanoverTagger as ht
import codecs
import spacy

In [106]:
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct
def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopword]
    return text


def clean_text(text):
    text = text.lower()
    text = "".join([word for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in stopword]
    return text
def lemmatizing(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100
def create_word_map(lemmatized):
    word_index_map = {}
    current_index = 0
    index_word_map = []
    
    for tokens in lemmatized:
        for token in tokens:
            if token not in word_index_map:
                    word_index_map[token] = current_index
                    current_index += 1
                    index_word_map.append(token)
    return word_index_map,index_word_map

def tokens_to_vector(tokens,data=df,tokenized_column='lemmatized'):
    lem = list(data[tokenized_column])
    word_index_map,index_word_map = create_word_map(lem)
    x = np.zeros(len(word_index_map))
    for t in tokens:
        i = word_index_map[t]
        x[i] = 1
    return x

def visualize_data(data,tokenized_column):
    lem = list(data[tokenized_column])
    word_index_map,index_word_map = create_word_map(lem)
    
    N = len(lem)
    D = len(word_index_map)
    X = np.zeros((D, N))
    i = 0
    for tokens in lem:
        X[:,i] = tokens_to_vector(tokens,data,tokenized_column)
        i += 1
    
    svd = TruncatedSVD()
    Z = svd.fit_transform(X)
    
    dz = pd.DataFrame()
    dz['Words'] = word_index_map
    dz['z0'] = Z[:,0] 
    dz['z1'] = Z[:,1] 
    
    fig = px.scatter(dz, x="z0", y="z1", text="Words")

    fig.update_traces(textposition='top center')

    fig.update_layout(
        height=800,
        title_text='Scatterplot of the data'
    )

    fig.show()

In [107]:
visualize_data(df,'tokenized')